In [ ]:
import sys
sys.path.append("../")
import importlib

from src.fourier_wrapper import FourierWrapper
from src.display import (display_image_differences,
                         display_image_metrics)
from src.process import (virtual_coil_combination,
                         virtual_coil_combination_gpu,
                         adaptive_coil_combination,
                         adaptive_coil_combination_gpu)
from src.utils import (_DATABASE,
                       find_shifts,
                       create_mask,
                       shift_kspace,
                       generate_path,
                       load_metadata,
                       low_pass_filter,
                       load_kspace_data,
                       load_kspace_locations,
                       load_phase_from_dicom,
                       load_magnitude_from_dicom,
                       load_field_map_from_dicom)

from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold, GroupLASSO
from modopt.math.metrics import ssim, psnr, mse
from mri.operators import FFT, WaveletN, OWL
from mri.operators.fourier.non_cartesian import Stacked3DNFFT, NonCartesianFFT
from mri.operators.fourier.utils import estimate_density_compensation
from mri.reconstructors import SingleChannelReconstructor
from mri.reconstructors.utils.extract_sensitivity_maps import get_Smaps

import h5py
import matplotlib.pyplot as plt
import numpy as np
import numpy.fft as nf
import os
import os.path as op

from skimage.morphology import convex_hull_image

# Options

In [ ]:
#"200310_invivo_7T_SPA" #"200310_invivo_7T_SPI"

#"200626_invivo_3T_CYL" #"200626_invivo_3T_SPH" #"200626_invivo_3T_F3D"

#"200717_invivo_3T_SHS" #"200717_invivo_3T_DSS"

#"201210_invivo_3T_S10" #"201210_invivo_3T_S20" #"201210_invivo_3T_F10" #"201210_invivo_3T_F20"

#"210723_invivo_3T_C15" #"210723_invivo_3T_C20" #"210723_invivo_3T_TW8"

#"211020_invivo_3T_SPA"

content = {
    # Folders
    "data_folder": "/volatile/daval/Data",#"/neurospin/optimed/GuillaumeDavalFrerot/Mondor", #
    "output_folder": "../out",
    # Reconstruction setup
    "acquisition": "211020_invivo_3T_SPA",
    "reconstructor": "single_channel",
    "regularizer": "SparseThreshold", # "GroupLASSO" # "OWL"
    "combination": "NOC",
    "nb_channels": 1,
    "nb_iterations": 50,
    # Correction setup
    "correction": False,
    # Extra args
    "comment": "",
}

load_metadata(content)

In [ ]:
nb_adc_samples = content["nb_adc_samples"]
OSF = content["oversampling_factor"]
resolution = content["resolution"]
N, _, Nz = content["matrix_size"]
shifts = content["shifts"]
Te = content["echo_time"]

ftype, ctype = np.float32, np.complex64
nb_adc_samples = nb_adc_samples * OSF
dwell_time = 10e-6 / OSF
alpha = 1e-1
beta = 0

# Loading

In [ ]:
# 201210
if (content["acquisition"][:6] == "201210"):
    mag_foldername = "/volatile/daval/Data/201210_invivo_3T/dcm/MagImages"
    pha_foldername = "/volatile/daval/Data/201210_invivo_3T/dcm/PhaImages"
# 211020
elif (content["acquisition"][:6] == "211020"):
    mag_foldername = "/volatile/daval/Data/211020_invivo_3T/dcm/Cartesian_MAG"
    pha_foldername = "/volatile/daval/Data/211020_invivo_3T/dcm/Cartesian_PHA"

magnitude = load_magnitude_from_dicom(mag_foldername)
phase = load_phase_from_dicom(pha_foldername)
volume = magnitude * np.exp(1j * phase)

In [ ]:
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(volume[...,i]), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(volume[...,i,:].T), cmap="gray")
    ax2.imshow(np.angle(volume[...,i,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(volume[i,:,:].T), cmap="gray")
    ax2.imshow(np.angle(volume[i,:,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

# Projection

In [ ]:
# Cylindrical Stack-of-SPARKLING
#filename = "/volatile/daval/Data/200626_invivo_3T/traj/01_dim2_SOSnoVDS_N384x384x208_FOV0.23x0.23x0.1248_Nc64_Ns2049.0_cutoff12_decay2.bin"

# Spherical Stack-of-SPARKLING
#filename = "/volatile/daval/Data/200626_invivo_3T/traj/02_dim3_sphereSOS_N384x384x208_FOV0.23x0.23x0.1248_Nc40_Ns2049.0_cutoff12_decay2.bin"

# Full3D SPARKLING
#filename = op.join(content["data_folder"], content["acquisition"][:-4], "traj", content["trajectory"])
#filename = "/volatile/daval/Data/mondor/traj/dim3_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3_densNone_P0.75_OSF5_smax140e-3__D9M4Y2021T1044.bin"
#filename = "/volatile/daval/Data/mondor/traj/dim3_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c20_d2_densNone_P0.75_OSF5_smax140e-3__D9M4Y2021T1048.bin"
#filename = "/volatile/daval/Data/mondor/traj/dim3_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3_denslog_fourier.npy_P0.75_OSF5_smax140e-3__D9M4Y2021T1058.bin"
#filename = "/volatile/daval/Data/mondor/traj/dim3_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3_P0.75_OSF_S140e-3__D9M4Y2021T1044.bin"

 # SNOREKLING
#basename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/TemporalWeights/N384x384x208_Nc70_TemporalWeights"
#filename = op.join(basename, "dim3_i_RadialIO_P0.75_TW-1_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T1452.bin")
#filename = op.join(basename, "dim3_i_RadialIO_P0.75_TW1_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T1455.bin")
#filename = op.join(basename, "dim3_i_RadialIO_P0.75_TW2_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T150.bin")
#filename = op.join(basename, "dim3_i_RadialIO_P0.75_TW4_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T155.bin")

# SNOREKLING REPROJECT
#filename = "/volatile/daval/Data/210917_phantom_3T/traj/01_dim3_i_RadialIO_P0.75_TW-1_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T1452_reproject.bin"
#filename = "/volatile/daval/Data/210917_phantom_3T/traj/02_dim3_i_RadialIO_P0.75_TW1_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T1455_reproject.bin"
#filename = "/volatile/daval/Data/210917_phantom_3T/traj/03_dim3_i_RadialIO_P0.75_TW2_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T150_reproject.bin"
#filename = "/volatile/daval/Data/210917_phantom_3T/traj/04_dim3_i_RadialIO_P0.75_TW4_N384x384x208_FOV0.23x0.23x0.1248_Nc70_Ns2048_c15_d3__D17M6Y2021T155_reproject.bin"

# SNOREKLING RECENT
#filename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/TemporalWeights/C25D2AF15/dim3_i_RadialIO_P0.75_TW-1_N384x384x208_FOV0.23x0.23x0.1248_Nc73_Ns2048_c25_d2__D6M10Y2021T225_reproject.bin"
#filename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/TemporalWeights/C25D2AF15/dim3_i_RadialIO_P0.75_TW1_N384x384x208_FOV0.23x0.23x0.1248_Nc73_Ns2048_c25_d2__D6M10Y2021T225_reproject.bin"
#filename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/TemporalWeights/C25D2AF15/dim3_i_RadialIO_P0.75_TW2_N384x384x208_FOV0.23x0.23x0.1248_Nc73_Ns2048_c25_d2__D6M10Y2021T730_reproject.bin"
#filename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/TemporalWeights/C25D2AF15/dim3_i_RadialIO_P0.75_TW4_N384x384x208_FOV0.23x0.23x0.1248_Nc73_Ns2048_c25_d2__D6M10Y2021T225_reproject.bin"

# MUCH MORE SPARKLING
basename = "/neurospin/optimed/Chaithya/Trajectories/SPARKLING/Full3D/MUCH"
filename = op.join(basename, "MUCH/dim3_i_CartesianLow_P0.75_N384x384x208_FOV0.23x0.23x0.1248_Nc72_Ns1005_c18.75_d1__D7M2Y2022T1237.bin")
#filename = op.join(basename, "MORE/dim3_i_RadialIO_P0.75_TW1_N384x384x208_FOV0.23x0.23x0.1248_Nc72_Ns2048_c18.75_d1__D7M2Y2022T932.bin")
#filename = op.join(basename, "MORE/dim3_i_RadialIO_P0.75_TW-1_N384x384x208_FOV0.23x0.23x0.1248_Nc72_Ns2048_c18.75_d1__D7M2Y2022T932.bin")

nb_adc_samples = 5 * 2046 # FIXME
kspace_locations = load_kspace_locations(filename, dwell_time * 1e3, nb_adc_samples, kmax=1/(2*6e-4))
kspace_locations = kspace_locations.astype(ftype)

In [ ]:
print("K-space locations shape: {}".format(kspace_locations.shape))

nb_shots = int(kspace_locations.shape[0] / nb_adc_samples)
for i in range(0, nb_shots, nb_shots // 50):
    plt.plot(kspace_locations[i * nb_adc_samples:(i+1) * nb_adc_samples, 0],
             kspace_locations[i * nb_adc_samples:(i+1) * nb_adc_samples, 1])
plt.show()

In [ ]:
density_weights = estimate_density_compensation(kspace_locations, volume.shape, num_iterations=10)

In [ ]:
fourier_op = NonCartesianFFT(samples=kspace_locations, shape=(N, N, Nz), n_coils=1,
                             implementation="gpuNUFFT", density_comp=density_weights)

In [ ]:
new_kspace = np.ascontiguousarray(fourier_op.op(np.ascontiguousarray(volume)))
print(new_kspace.shape)

In [ ]:
print(np.mean(volume))
print(np.mean(new_kspace))
print(new_kspace.shape)

# Reconstruction

In [ ]:
if (alpha == 0):
    linear_op = Identity()
    linear_op.n_coils = 1
else:
    linear_op = WaveletN(
        wavelet_name='sym8',
        nb_scale=3,
        n_coils=kspace_data.shape[0] if (content["reconstructor"] == "calibrationless") else 1,
        padding_mode='periodization',
        dim=3,
        n_jobs=-1,
    )

In [ ]:
if (alpha == 0):
    regularizer_op = Identity()
    regularizer_op.cost = lambda x: 0
    regularizer_op.weights = 0
else:
    if (content["regularizer"] == "OWL"):
        linear_op.op(np.zeros((kspace_data.shape[0], N, N, Nz))) # force to setup coeffs_shape in linear_op
        regularizer_op = OWL(alpha=alpha, beta=beta, bands_shape=linear_op.coeffs_shape, n_coils=kspace_data.shape[0], mode="band_based", n_jobs=-1)
    elif (content["regularizer"] == "GroupLASSO"):
        regularizer_op = GroupLASSO(alpha)
    else:
        regularizer_op = SparseThreshold(Identity(), alpha, thresh_type="soft")

In [ ]:
reconstructor_args = {
    "fourier_op": fourier_op,
    "linear_op": linear_op,
    "regularizer_op": regularizer_op,
    "gradient_formulation": 'synthesis',
    "verbose": 100,
    "lipschitz_cst": None,
    "num_check_lips": 0,
    "lips_calc_max_iter": 15,
}

reconstructor = SingleChannelReconstructor(**reconstructor_args)

In [ ]:
print(np.mean(volume))
print(np.mean(new_kspace))
print(new_kspace.shape)

In [ ]:
reconstruct_args = {
    "kspace_data": new_kspace[None, ...],
    "optimization_alg": 'fista',
    "num_iterations": content["nb_iterations"],
}

new_volume, _, _ = reconstructor.reconstruct(**reconstruct_args)

In [ ]:
print(np.mean(volume))
print(np.mean(new_kspace))
print(new_kspace.shape)

In [ ]:
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(new_volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(new_volume[...,i]), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(new_volume[...,i,:].T), cmap="gray")
    ax2.imshow(np.angle(new_volume[...,i,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(new_volume[i,:,:].T), cmap="gray")
    ax2.imshow(np.angle(new_volume[i,:,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

# Reconstruction with 0th order T2* compensation

In [ ]:
time_vec = dwell_time * np.arange(nb_adc_samples)
echo_time = Te - dwell_time * nb_adc_samples / 2
time_vec = (time_vec + echo_time).astype(ftype)
time_vec = np.tile(time_vec, new_kspace.shape[0] // time_vec.shape[0])

T2s = 30e-3

In [ ]:
%%script false --no-raise-error
reconstruct_args = {
    "kspace_data": new_kspace * np.exp(-time_vec / T2s),
    "optimization_alg": 'fista',
    "num_iterations": content["nb_iterations"],
}

t2_volume, _, _ = reconstructor.reconstruct(**reconstruct_args)

In [ ]:
%%script false --no-raise-error
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(t2_volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(t2_volume[...,i]), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
%%script false --no-raise-error
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(t2_volume[...,i,:].T), cmap="gray")
    ax2.imshow(np.angle(t2_volume[...,i,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
%%script false --no-raise-error
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(t2_volume[i,:,:].T), cmap="gray")
    ax2.imshow(np.angle(t2_volume[i,:,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

# B0 effects addition

In [ ]:
mask = create_mask(volume)
mask = np.stack(list(map(convex_hull_image, np.transpose(mask, [2, 0, 1]))), axis=-1)

In [ ]:
content["path_to_field_map"] = op.join(content["data_folder"],
                                           content["acquisition"][:-4],
                                           "dcm", content["b0map"])

field_map, range_w = load_field_map_from_dicom(content["path_to_field_map"], N, N, Nz, unwrap="old")

for z in range(Nz):
    field_map[...,z] = np.rot90(field_map[...,z])

field_map = field_map * mask
field_map = low_pass_filter(field_map).real.astype(ftype)

In [ ]:
from skimage.restoration import unwrap_phase

def update_field_map(volume, field_map=None):
    if (field_map is None):
        field_map = np.zeros(volume.shape).astype(ftype)
    
    low_freq_magn  = np.abs(volume)
    low_freq_phase = np.angle(volume)

    low_freq_mask = create_mask(low_freq_magn)
    low_freq_mask = np.stack(list(map(convex_hull_image, np.transpose(low_freq_mask, [2, 0, 1]))), axis=-1)
    
    #poisson_field_map = poisson_unwrap_gpu(low_freq_phase * low_freq_mask, low_freq_magn * low_freq_mask, kmax=50)
    poisson_field_map = unwrap_phase(low_freq_phase * low_freq_mask)
    poisson_field_map = poisson_field_map - (np.mean(np.angle(np.exp(1j * low_freq_phase)
                                                            / np.exp(1j * poisson_field_map))[np.where(low_freq_mask)]))
    
    mask = np.where(low_freq_mask)
    shift = int(np.around(1 / Te))
    #poisson_field_map = poisson_field_map - (2 * np.pi) * np.around(np.mean(poisson_field_map[mask]) / (2 * np.pi))
    poisson_field_map -= np.median(poisson_field_map[mask])

    field_map = field_map + (poisson_field_map * low_freq_mask / (2 * np.pi * Te))
    field_map -= np.median(field_map[mask])
    #field_map = field_map - shift * np.around(np.mean(field_map[mask]) / shift) # FIXME

    field_map = low_pass_filter(field_map * low_freq_mask).real.astype(ftype)
    return field_map, low_freq_mask

In [ ]:
#field_map, mask = update_field_map(volume) # FIXME

In [ ]:
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2, ax3) = fig.subplots(1, 3, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(volume[...,i]), cmap="gray")
    ax3.imshow(field_map[...,i], cmap="jet", vmin=-250, vmax=250)

    ax1.axis("off")
    ax2.axis("off")
    ax3.axis("off")
    plt.show()

In [ ]:
time_vec = dwell_time * np.arange(nb_adc_samples)
echo_time = Te - dwell_time * nb_adc_samples / 2
time_vec = (time_vec + echo_time).astype(ftype)

In [ ]:
ifourier_op = FourierWrapper(fourier_op, field_map, time_vec, mask, n_bins=1000, # FIXME +-
                             L=int(3 + (np.max(field_map) - np.min(field_map)) // 25),
                             coefficients="svd", weights="full")
ifourier_op.display_infos()

In [ ]:
b0_kspace = ifourier_op.op(np.abs(volume))
time_vec = np.tile(time_vec, new_kspace.shape[0] // time_vec.shape[0])
print(b0_kspace.shape)

# Reconstruction with B0

In [ ]:
reconstruct_args = {
    "kspace_data": b0_kspace * np.exp(time_vec),
    "optimization_alg": 'fista',
    "num_iterations": content["nb_iterations"],
}

b0_volume, _, _ = reconstructor.reconstruct(**reconstruct_args)

In [ ]:
print("Norm:\t{}".format(np.linalg.norm(b0_volume * mask)))
print("MSE:\t{}".format(mse(b0_volume, volume, mask)))
print("PSNR:\t{}".format(psnr(b0_volume, volume, mask)))
print("SSIM:\t{}".format(ssim(b0_volume, volume, mask)))

In [ ]:
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(b0_volume[...,i]), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[...,i,:].T), cmap="gray")
    ax2.imshow(np.angle(b0_volume[...,i,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[i,:,:].T), cmap="gray")
    ax2.imshow(np.angle(b0_volume[i,:,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
slia = 160
slic = 384 - 165 # FIXME
slis = 205 # FIXME
vmin = 0
vmax = np.max(np.abs(new_volume)) / 2

name = "MUCH_BETTER"
basename = "../MMS"
vol = b0_volume

plt.figure(figsize=(10,10))
plt.imshow(np.abs(vol[:,:,slia]).T, vmin=vmin, vmax=vmax, cmap="gray")
plt.show()

plt.figure(figsize=(10,10))
plt.imshow(np.abs(vol[:,slis,:]).T, vmin=vmin, vmax=vmax, cmap="gray")
plt.show()

plt.figure(figsize=(10,10))
plt.imshow(np.abs(vol[slic,:,:]).T, vmin=vmin, vmax=vmax, cmap="gray")
plt.show()

plt.imsave(op.join(basename, "retrospective_invivo_axial_{}.png".format(name)), np.abs(vol[:,:,slia]), cmap="gray", vmin=vmin, vmax=vmax)
plt.imsave(op.join(basename, "retrospective_invivo_sagital_{}.png".format(name)), np.abs(vol[:,slis,:]), cmap="gray", vmin=vmin, vmax=vmax)
plt.imsave(op.join(basename, "retrospective_invivo_coronal_{}.png".format(name)), np.abs(vol[slic,:,:]), cmap="gray", vmin=vmin, vmax=vmax)

In [ ]:
stop

# Correction with B0

In [ ]:
ifourier_op = FourierWrapper(fourier_op, -field_map, time_vec, mask, n_bins=1000,
                             L=int(3 + (np.max(field_map) - np.min(field_map)) // 25),
                             coefficients="svd", weights="full")
ifourier_op.display_infos()

In [ ]:
reconstructor_args = {
    "fourier_op": ifourier_op,
    "linear_op": linear_op,
    "regularizer_op": regularizer_op,
    "gradient_formulation": 'synthesis',
    "verbose": 1,
    "lipschitz_cst": None,
    "num_check_lips": 0,
    "lips_calc_max_iter": 5,
}

reconstructor = SingleChannelReconstructor(**reconstructor_args)

In [ ]:
reconstruct_args = {
    "kspace_data": b0_kspace,
    "optimization_alg": 'fista',
    "num_iterations": 10,
}

b0_volume, _, _ = reconstructor.reconstruct(**reconstruct_args)

In [ ]:
for i in range(0, Nz, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[...,i]), cmap="gray")
    ax2.imshow(np.angle(b0_volume[...,i]), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[...,i,:].T), cmap="gray")
    ax2.imshow(np.angle(b0_volume[...,i,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()

In [ ]:
for i in range(0, N, 10):
    fig = plt.figure(figsize=(10, 10), dpi=200)
    (ax1, ax2) = fig.subplots(1, 2, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0})
    ax1.imshow(np.abs(b0_volume[i,:,:].T), cmap="gray")
    ax2.imshow(np.angle(b0_volume[i,:,:].T), cmap="gray")

    ax1.axis("off")
    ax2.axis("off")
    plt.show()